In [ ]:
import pandas as pd
import json

In [4]:
# Read data for evaluation, csv still contains ground_truth in caption column
df = pd.read_csv('paired_dataset/paired_dataset_gpt.csv') 

hand_curated = [0, 5, 13, 15, 19, 30, 37, 41, 82, 91, 103, 114, 124, 139, 202, 436, 441, 455, 588, 614, 966, 971, 997, 1342]
df_minus_24 = df.drop(index=hand_curated)
# print(df_minus_24.head(10))
# Select rows where 'is_balanced_subset' is False
df_train = df_minus_24[df_minus_24['is_balanced_subset'] == False]
df_test = df_minus_24[df_minus_24['is_balanced_subset'] == True]
print("Length of df_train: ", len(df_train))
print("Length of df_test: ", len(df_test))

Length of df_train:  4498
Length of df_test:  999


In [5]:
df.columns

Index(['ytid', 'start_s', 'end_s', 'audioset_positive_labels', 'aspect_list',
       'caption', 'author_id', 'is_balanced_subset', 'is_audioset_eval',
       'novice', 'gen_expert'],
      dtype='object')

In [6]:
df[['novice','gen_expert','is_balanced_subset']]

,novice,gen_expert,is_balanced_subset
0,A melancholic piano song with a female singer ...,A somber piano ballad featuring ethereal femal...,False
1,"A chill song with electric guitar, simple beat...",A laid-back acoustic music piece with a clean ...,False
2,A man singing a catchy tune while snapping his...,A jazz vocalist performs a lively piece with f...,False
3,"A catchy song with guitars, bass, piano, and t...",This is a commercial jingle featuring a blend ...,False
4,A fun folk song played on a rubber instrument ...,A lively folk music composition played on a ru...,False
...,...,...,...
5516,Someone playing a wooden drum box.,A percussionist performing on a cajón.,False
5517,An ancient Arabian folk instrumental with stri...,An ancient Arabian folk instrumental featuring...,True
5518,An electrifying rock guitar solo with lots of ...,A high-energy rock guitar solo featuring virtu...,True
5519,A slow and emotional instrumental piece with d...,An instrumental piece with a melancholic atmos...,False


In [14]:
def parse_instruct_format(system, user, assistant, isVal):
    parsed = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    if isVal:
        return parsed
    else:
        parsed += f'{assistant}<|eot_id|>'
        return parsed

In [17]:
print(parse_instruct_format('system_test', 'user_test', 'assistant_test', False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

system_test<|eot_id|><|start_header_id|>user<|end_header_id|>

user_test<|eot_id|><|start_header_id|>assistant<|end_header_id|>

assistant_test<|eot_id|>


In [8]:
system_prompt = "You are a helpful assistant that converts novice-friendly music descriptions into expert descriptions.\n"
system_prompt += "Transform the given input novice-level prompt into a prompt that a user with extensive music training and terminologies would write.\n"
system_prompt += "Keep the instruments, genres, mood, and other information that represents the essence of the music. "
system_prompt += "Use exactly 2 sentences."

In [18]:
parsed_instruct_dataset_train = []
for index, row in df_train.iterrows():
    instruct = parse_instruct_format(system_prompt, row['novice'], row['caption'], False)
    parsed_instruct_dataset_train.append({'text':f'{instruct}'})

In [19]:
with open("instruct_dataset_train.jsonl", "w") as file:
    for item in parsed_instruct_dataset_train:
        file.write(json.dumps(item) + "\n")

In [20]:
parsed_instruct_dataset_validation = []
for index, row in df_test.iterrows():
    instruct = parse_instruct_format(system_prompt,row['novice'], row['caption'], True)
    parsed_instruct_dataset_validation.append({'text':f'{instruct}'})

In [21]:
with open("instruct_dataset_validation.jsonl", "w") as file:
    for item in parsed_instruct_dataset_validation:
        file.write(json.dumps(item) + "\n")